In [ ]:
# Comando Backup Bando de Dados SQL Server .bak para restore

# Dentro do container SQL Server
docker exec -it etl_sqlserver bash

BACKUP DATABASE N'projeto_etl_preditivo'
TO DISK = N'/var/opt/mssql/backups/projeto_etl_preditivo.bak'
WITH INIT, STATS = 10;
GO

# OU

# Comando via docker exec
docker exec etl_sqlserver /opt/mssql-tools/bin/sqlcmd \
   -S localhost -U sa -P "YourStrong@Passw0rd" \
   -Q "BACKUP DATABASE [projeto_etl_preditivo] TO DISK='/var/opt/mssql/backup/projeto_etl_preditivo.bak' WITH INIT, STATS=10"



In [4]:
import pyodbc
from dotenv import load_dotenv
import os

load_dotenv()

conn = pyodbc.connect(    
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={os.getenv('MSSQL_HOST')};"
    f"DATABASE={os.getenv('MSSQL_DB')};"
    f"UID={os.getenv('MSSQL_USER')};"
    f"PWD={os.getenv('MSSQL_SA_PASSWORD')};"
    "Encrypt=no;"
    "TrustServerCertificate=yes;",
     autocommit=False  
)
cur = conn.cursor()
cur.execute("SELECT name FROM sys.databases;")
print(cur.fetchone())
conn.close()

('master',)


In [ ]:
# DataFrame SQL Server - vw_Recompra90Dias
import pyodbc
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

conn = pyodbc.connect(    
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={os.getenv('MSSQL_HOST')};"
    f"DATABASE={os.getenv('MSSQL_DB')};"
    f"UID={os.getenv('MSSQL_USER')};"
    f"PWD={os.getenv('MSSQL_SA_PASSWORD')};"
    "Encrypt=no;"
    "TrustServerCertificate=yes;",
     autocommit=False  
)

# Criar DataFrame diretamente da consulta
df_transacoes = pd.read_sql("SELECT * FROM dw.vw_Recompra90Dias", conn)

print(f"DataFrame criado com {len(df_transacoes)} registros")
display(df_transacoes)

conn.close()

/tmp/ipykernel_5222/265892726.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_transacoes = pd.read_sql("SELECT * FROM dw.vw_Recompra90Dias", conn)


DataFrame criado com 321720 registros


,ClienteKey,DateKey,DataVenda,Valor,Recompra90Dias
0,924,20170520,2017-05-20,900.00,1
1,784,20150514,2015-05-14,8400.00,1
2,456,20150909,2015-09-09,178.00,1
3,544,20151007,2015-10-07,5500.00,1
4,343,20150825,2015-08-25,297.00,1
...,...,...,...,...,...
321715,6011,20250930,2025-09-30,390.70,0
321716,6011,20250930,2025-09-30,8739.39,0
321717,6011,20250930,2025-09-30,1131.34,0
321718,6011,20250930,2025-09-30,254.52,0


In [8]:
import os
import pandas as pd
from datetime import datetime

OUTPUT_DIR = "./data_slices_csv"
os.makedirs(OUTPUT_DIR, exist_ok=True)

CURRENT_YEAR = datetime.now().year
LAST_N = int(os.getenv("SLICE_LAST_N_YEARS", 3))
MIN_YEAR = CURRENT_YEAR - LAST_N + 1    

df_transacoes["DataVenda"] = pd.to_datetime(df_transacoes["DataVenda"], errors="coerce")
df = df_transacoes[df_transacoes["DataVenda"].dt.year >= MIN_YEAR]

out = f"{OUTPUT_DIR}/recompra.csv"
df.to_csv(out, index=False)
print(f"[OK] Salvo {len(df):,} linhas em {out}")

[OK] Salvo 121,372 linhas em ./data_slices_csv/recompra.csv


In [17]:
# Salvar no hdfs de csv para formato parquet
import os
import pandas as pd
from datetime import datetime
import pyarrow as pa
import pyarrow.parquet as pq

# INPUT_CSV = "./data_slices_csv/recompra.csv"

from hdfs import InsecureClient
client = InsecureClient('http://localhost:9870', user='root')
print(client.list('/'))


['datalake', 'teste.txt', 'teste_local.txt']


In [7]:
from hdfs import InsecureClient
client = InsecureClient("http://localhost:9870", user="root")

# client.write("/teste_local2.txt", data=b"ok", overwrite=True)
print(client.list("/"))


['datalake', 'teste.txt', 'teste_local.txt']


In [4]:
from hdfs import InsecureClient
import traceback

HDFS_URL = "http://localhost:9870"  # URL do NameNode
HDFS_USER = "root"  # Pode ser outro usuário, se configurado

try:
    # Cria o cliente para acessar o HDFS
    client = InsecureClient(f"{HDFS_URL}", user=HDFS_USER)

    # Tenta listar a raiz do HDFS
    files = client.list('/')
    print("HDFS está ativo! Itens no HDFS root:")
    for f in files:
        print(f"- {f}")

except Exception as e:
    print("❌ Não foi possível conectar ao HDFS")
    print("Erro:")
    traceback.print_exc()


HDFS está ativo! Itens no HDFS root:
- datalake
- teste.txt


In [1]:
# Carga DimCategoria

from dotenv import load_dotenv
import os
import psycopg2
import pyodbc

load_dotenv()

pg = psycopg2.connect(
    host=os.getenv("POSTGRES_DATADT_HOST"),
    dbname=os.getenv("POSTGRES_DATADT_DB"),
    user=os.getenv("POSTGRES_DATADT_USER"),
    password=os.getenv("POSTGRES_DATADT_PASSWORD"),
)
pg_cur = pg.cursor()

sql = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={os.getenv('MSSQL_HOST')};"
    f"DATABASE={os.getenv('MSSQL_DB')};"
    f"UID={os.getenv('MSSQL_USER')};"
    f"PWD={os.getenv('MSSQL_SA_PASSWORD')};"
    "Encrypt=no;"
    "TrustServerCertificate=yes;"
)
sql_cur = sql.cursor()


def get_max_id(sql_cur, table, column):
    sql_cur.execute(f"SELECT ISNULL(MAX({column}), 0) FROM {table}")
    return sql_cur.fetchone()[0]

def load_dim_categoria():
    print("Carregando DimCategoria...")

    last_id = get_max_id(sql_cur, "dw.DimCategoria", "CategoriaID")

    pg_cur.execute("""
        SELECT id, descricao
        FROM vendas.categoria
        WHERE id > %s
        ORDER BY id
    """, (last_id,))

    rows = pg_cur.fetchall()

    for r in rows:
        sql_cur.execute("""
            INSERT INTO dw.DimCategoria (CategoriaID, Descricao)
            VALUES (?, ?)
        """, (r[0], r[1]))

    sql.commit()
    print(f"{len(rows)} registros inseridos em DimCategoria.")

    
load_dim_categoria()

# 🔥 Fechando cursores e conexões corretamente
pg_cur.close()
pg.close()

sql_cur.close()
sql.close()


Carregando DimCategoria...
0 registros inseridos em DimCategoria.


In [11]:
# Carga DimCliente

from dotenv import load_dotenv
import os
import psycopg2
import pyodbc

load_dotenv()

# Conexão PostgreSQL
pg = psycopg2.connect(
    host=os.getenv("POSTGRES_DATADT_HOST"),
    dbname=os.getenv("POSTGRES_DATADT_DB"),
    user=os.getenv("POSTGRES_DATADT_USER"),
    password=os.getenv("POSTGRES_DATADT_PASSWORD"),
)
pg_cur = pg.cursor()

# Conexão SQL Server (DW)
sql = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={os.getenv('MSSQL_HOST')};"
    f"DATABASE={os.getenv('MSSQL_DB')};"
    f"UID={os.getenv('MSSQL_USER')};"
    f"PWD={os.getenv('MSSQL_SA_PASSWORD')};"
    "Encrypt=no;"
    "TrustServerCertificate=yes;"
)
sql_cur = sql.cursor()


def get_max_id(sql_cur, table, column):
    sql_cur.execute(f"SELECT ISNULL(MAX({column}), 0) FROM {table}")
    return sql_cur.fetchone()[0]


def load_dim_cliente():
    print("Carregando DimCliente...")

    # Último cliente carregado no DW
    last_id = get_max_id(sql_cur, "dw.DimCliente", "ClienteID")

    # Busca apenas clientes novos no PostgreSQL
    pg_sql = """
        SELECT DISTINCT id_cliente, cliente, tipo_pessoa
        FROM (
            SELECT pf.id AS id_cliente, 
                   pf.nome AS cliente,
                   'Pessoa Física'::text AS tipo_pessoa
            FROM vendas.nota_fiscal nf
            JOIN geral.pessoa p ON p.id = nf.id_cliente
            JOIN geral.pessoa_fisica pf ON pf.id = p.id
            GROUP BY pf.id, pf.nome

            UNION ALL

            SELECT pj.id AS id_cliente, 
                   pj.razao_social AS cliente,
                   'Pessoa Jurídica'::text AS tipo_pessoa
            FROM vendas.nota_fiscal nf
            JOIN geral.pessoa p ON p.id = nf.id_cliente
            JOIN geral.pessoa_juridica pj ON pj.id = p.id
            GROUP BY pj.id, pj.razao_social
        ) tb
        WHERE id_cliente > %s
        ORDER BY id_cliente ASC;
    """

    pg_cur.execute(pg_sql, (last_id,))
    rows = pg_cur.fetchall()

    # Insere no SQL Server
    for r in rows:
        sql_cur.execute("""
            INSERT INTO dw.DimCliente (ClienteID, Nome, Tipo_Pessoa)
            VALUES (?, ?, ?)
        """, (r[0], r[1], r[2]))

    sql.commit()
    print(f"{len(rows)} clientes inseridos em DimCliente.")


# Executa carga
load_dim_cliente()

# Fecha conexões
pg_cur.close()
pg.close()
sql_cur.close()
sql.close()


Carregando DimCliente...
15924 clientes inseridos em DimCliente.


In [13]:
# Carga DimFomaPagamento

from dotenv import load_dotenv
import os
import psycopg2
import pyodbc

load_dotenv()

# Conexão PostgreSQL
pg = psycopg2.connect(
    host=os.getenv("POSTGRES_DATADT_HOST"),
    dbname=os.getenv("POSTGRES_DATADT_DB"),
    user=os.getenv("POSTGRES_DATADT_USER"),
    password=os.getenv("POSTGRES_DATADT_PASSWORD"),
)
pg_cur = pg.cursor()

# Conexão SQL Server (DW)
sql = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={os.getenv('MSSQL_HOST')};"
    f"DATABASE={os.getenv('MSSQL_DB')};"
    f"UID={os.getenv('MSSQL_USER')};"
    f"PWD={os.getenv('MSSQL_SA_PASSWORD')};"
    "Encrypt=no;"
    "TrustServerCertificate=yes;"
)
sql_cur = sql.cursor()


def get_max_id(sql_cur, table, column):
    sql_cur.execute(f"SELECT ISNULL(MAX({column}), 0) FROM {table}")
    return sql_cur.fetchone()[0]

def load_dim_forma_pagamento():
    print("Carregando DimFormaPagamento...")

    last_id = get_max_id(sql_cur, "dw.DimFormaPagamento", "FormaPagamentoID")

    pg_cur.execute("""
        SELECT id, descricao
        FROM vendas.forma_pagamento
        WHERE id > %s
        ORDER BY id
    """, (last_id,))

    rows = pg_cur.fetchall()

    for r in rows:
        sql_cur.execute("""
            INSERT INTO dw.DimFormaPagamento (FormaPagamentoID, Descricao)
            VALUES (?, ?)
        """, (r[0], r[1]))

    sql.commit()
    print(f"{len(rows)} novos registros.")

load_dim_forma_pagamento()

# 🔥 Fechando cursores e conexões corretamente
pg_cur.close()
pg.close()
sql_cur.close()
sql.close()

Carregando DimFormaPagamento...
0 novos registros.


In [17]:
from database import get_postgres_cursor, get_sqlserver_cursor

def load_dim_fornecedor():
    print("\n=== Carregando DimFornecedor ===")

    # -----------------------------------------
    # Conexões
    # -----------------------------------------
    pg_conn, pg_cur = get_postgres_cursor()
    sql_conn, sql_cur = get_sqlserver_cursor()

    # -----------------------------------------
    # 1. Busca o maior ID já carregado no DW
    # -----------------------------------------
    sql_cur.execute("SELECT ISNULL(MAX(FornecedorID), 0) FROM dw.DimFornecedor;")
    last_id = sql_cur.fetchone()[0]

    print(f"Último FornecedorID carregado: {last_id}")

    # -----------------------------------------
    # 2. Busca novos fornecedores no Postgres
    # -----------------------------------------
    pg_query = """
        SELECT DISTINCT 
            a.id_fornecedor AS FornecedorID,
            b.razao_social AS Nome
        FROM vendas.produto a
        LEFT JOIN geral.pessoa_juridica b 
            ON a.id_fornecedor = b.id
        WHERE a.id_fornecedor IS NOT NULL
          AND a.id_fornecedor > %s
        ORDER BY a.id_fornecedor;
    """

    pg_cur.execute(pg_query, (last_id,))
    new_rows = pg_cur.fetchall()

    print(f"Encontrados {len(new_rows)} novos fornecedores.")

    # -----------------------------------------
    # 3. Insert no SQL Server
    # -----------------------------------------
    inserted = 0

    for fornecedor_id, nome in new_rows:
        sql_cur.execute("""
            INSERT INTO dw.DimFornecedor (
                FornecedorID, Nome
            )
            VALUES (?, ?)
        """, (fornecedor_id, nome))

        inserted += 1

    # -----------------------------------------
    # 4. Commit
    # -----------------------------------------
    sql_conn.commit()
    pg_conn.close()
    sql_conn.close()

    print(f"DimFornecedor - {inserted} registros inseridos com sucesso.")

try:
    load_dim_fornecedor()
except Exception as e:
    import traceback
    traceback.print_exc()



=== Carregando DimFornecedor ===
Último FornecedorID carregado: 15337
Encontrados 0 novos fornecedores.
DimFornecedor - 0 registros inseridos com sucesso.


In [18]:
# Carga DimNotaFiscal

from database import get_postgres_cursor, get_sqlserver_cursor

def load_dim_nota_fiscal():
    print("\n=== Carregando DimNotaFiscal ===")

    # -----------------------------------------
    # Conexões
    # -----------------------------------------
    pg_conn, pg_cur = get_postgres_cursor()
    sql_conn, sql_cur = get_sqlserver_cursor()

    # -----------------------------------------
    # 1. Busca o maior NotaFiscalID já carregado
    # -----------------------------------------
    sql_cur.execute("SELECT ISNULL(MAX(NotaFiscalID), 0) FROM dw.DimNotaFiscal;")
    last_id = sql_cur.fetchone()[0]

    print(f"Último NotaFiscalID carregado: {last_id}")

    # -----------------------------------------
    # 2. Busca novas notas fiscais no Postgres
    # -----------------------------------------
    pg_query = """
        SELECT 
            id AS NotaFiscalID,
            numero_nf AS NumeroNF,
            valor AS ValorTotal,
            data_venda
        FROM vendas.nota_fiscal
        WHERE id > %s
        ORDER BY id;
    """

    pg_cur.execute(pg_query, (last_id,))
    rows = pg_cur.fetchall()

    print(f"Encontrados {len(rows)} novos registros de Nota Fiscal.")

    # -----------------------------------------
    # 3. Insert no SQL Server
    # -----------------------------------------
    inserted = 0

    for nf_id, numero_nf, valor_total, data_venda in rows:

        # DateKey convertido para inteiro YYYYMMDD
        date_key = int(data_venda.strftime("%Y%m%d"))

        sql_cur.execute("""
            INSERT INTO dw.DimNotaFiscal (
                NotaFiscalID,
                NumeroNF,
                ValorTotal,
                DateKey
            )
            VALUES (?, ?, ?, ?)
        """, (nf_id, numero_nf, valor_total, date_key))

        inserted += 1

    # -----------------------------------------
    # 4. Commit e fechar conexões
    # -----------------------------------------
    sql_conn.commit()
    pg_conn.close()
    sql_conn.close()

    print(f"DimNotaFiscal - {inserted} registros inseridos com sucesso.")

load_dim_nota_fiscal()


=== Carregando DimNotaFiscal ===
Último NotaFiscalID carregado: 0
Encontrados 120621 novos registros de Nota Fiscal.
DimNotaFiscal - 120621 registros inseridos com sucesso.


In [19]:
# Carga DimParcela

from database import get_postgres_cursor, get_sqlserver_cursor

def load_dim_parcela():
    print("\n=== Carregando DimParcela ===")

    # -----------------------------------------
    # Conexões
    # -----------------------------------------
    pg_conn, pg_cur = get_postgres_cursor()
    sql_conn, sql_cur = get_sqlserver_cursor()

    # -----------------------------------------
    # 1. Buscar o maior ParcelaID já carregado no DW
    # -----------------------------------------
    sql_cur.execute("SELECT ISNULL(MAX(ParcelaID), 0) FROM dw.DimParcela;")
    last_id = sql_cur.fetchone()[0]

    print(f"Último ParcelaID carregado: {last_id}")

    # -----------------------------------------
    # 2. Buscar novas parcelas no Postgres
    # -----------------------------------------
    pg_query = """
        SELECT 
            id AS ParcelaID,
            id_nota_fiscal AS NotaFiscalID,
            numero AS NumeroParcela,
            valor AS ValorParcela,
            vencimento
        FROM vendas.parcela
        WHERE id > %s
        ORDER BY id;
    """

    pg_cur.execute(pg_query, (last_id,))
    rows = pg_cur.fetchall()

    print(f"Encontrados {len(rows)} novos registros de parcelas.")

    # -----------------------------------------
    # 3. Inserir no SQL Server
    # -----------------------------------------
    inserted = 0

    for parcela_id, nota_id, numero, valor, vencimento in rows:

        # Se a data estiver nula, define como 1900-01-01
        if vencimento is None:
            date_key = 19000101
        else:
            date_key = int(vencimento.strftime("%Y%m%d"))

        sql_cur.execute("""
            INSERT INTO dw.DimParcela (
                ParcelaID,
                NotaFiscalID,
                NumeroParcela,
                ValorParcela,
                Vencimento,
                DateKey
            )
            VALUES (?, ?, ?, ?, ?, ?)
        """, (parcela_id, nota_id, numero, valor, vencimento, date_key))

        inserted += 1

    # -----------------------------------------
    # 4. Commit e fechar
    # -----------------------------------------
    sql_conn.commit()
    pg_conn.close()
    sql_conn.close()

    print(f"DimParcela - {inserted} registros inseridos com sucesso.")

load_dim_parcela()


=== Carregando DimParcela ===
Último ParcelaID carregado: 0
Encontrados 987022 novos registros de parcelas.
DimParcela - 987022 registros inseridos com sucesso.


In [20]:
# Carga DimProduto

from database import get_postgres_cursor, get_sqlserver_cursor

def load_dim_produto():
    print("\n=== Carregando DimProduto ===")

    # -----------------------------------------
    # Conexões
    # -----------------------------------------
    pg_conn, pg_cur = get_postgres_cursor()
    sql_conn, sql_cur = get_sqlserver_cursor()

    # -----------------------------------------
    # 1. Buscar maior ProdutoID já carregado no DW
    # -----------------------------------------
    sql_cur.execute("SELECT ISNULL(MAX(ProdutoID), 0) FROM dw.DimProduto;")
    last_id = sql_cur.fetchone()[0]

    print(f"Último ProdutoID carregado: {last_id}")

    # -----------------------------------------
    # 2. Buscar novos produtos no Postgres
    # -----------------------------------------
    pg_query = """
        SELECT
            id AS ProdutoID,
            id_categoria AS CategoriaID,
            id_fornecedor AS FornecedorID,
            nome,
            valor_venda,
            valor_custo
        FROM vendas.produto
        WHERE id > %s
        ORDER BY id;
    """

    pg_cur.execute(pg_query, (last_id,))
    rows = pg_cur.fetchall()

    print(f"Encontrados {len(rows)} novos produtos.")

    # -----------------------------------------
    # 3. Inserção no SQL Server
    # -----------------------------------------
    inserted = 0

    for produto_id, categoria_id, fornecedor_id, nome, valor_venda, valor_custo in rows:

        # Obter CategoriaKey
        sql_cur.execute(
            "SELECT CategoriaKey FROM dw.DimCategoria WHERE CategoriaID = ?",
            (categoria_id,)
        )
        categoria_row = sql_cur.fetchone()
        categoria_key = categoria_row[0] if categoria_row else None

        # Obter FornecedorKey
        sql_cur.execute(
            "SELECT FornecedorKey FROM dw.DimFornecedor WHERE FornecedorID = ?",
            (fornecedor_id,)
        )
        fornecedor_row = sql_cur.fetchone()
        fornecedor_key = fornecedor_row[0] if fornecedor_row else None

        # Inserir produto
        sql_cur.execute("""
            INSERT INTO dw.DimProduto (
                ProdutoID,
                CategoriaKey,
                FornecedorKey,
                Nome,
                ValorVenda,
                ValorCusto
            )
            VALUES (?, ?, ?, ?, ?, ?)
        """, (
            produto_id,
            categoria_key,
            fornecedor_key,
            nome,
            valor_venda,
            valor_custo
        ))

        inserted += 1

    # -----------------------------------------
    # 4. Commit e fechar conexões
    # -----------------------------------------
    sql_conn.commit()
    pg_conn.close()
    sql_conn.close()

    print(f"DimProduto - {inserted} registros inseridos com sucesso.")

load_dim_produto()


=== Carregando DimProduto ===
Último ProdutoID carregado: 0
Encontrados 233 novos produtos.
DimProduto - 233 registros inseridos com sucesso.


In [21]:
# Carga DimSituacaoTitulo

from database import get_postgres_cursor, get_sqlserver_cursor

def load_dim_situacao_titulo():
    print("\n=== Carregando DimSituacaoTitulo ===")

    # -----------------------------------------
    # Conexões
    # -----------------------------------------
    pg_conn, pg_cur = get_postgres_cursor()
    sql_conn, sql_cur = get_sqlserver_cursor()

    # -----------------------------------------
    # 1. Buscar o maior SituacaoTituloID já carregado
    # -----------------------------------------
    sql_cur.execute("SELECT ISNULL(MAX(SituacaoTituloID), 0) FROM dw.DimSituacaoTitulo;")
    last_id = sql_cur.fetchone()[0]

    print(f"Último SituacaoTituloID carregado: {last_id}")

    # -----------------------------------------
    # 2. Buscar novos registros no Postgres
    # -----------------------------------------
    pg_query = """
        SELECT 
            id AS SituacaoTituloID,
            descricao AS Descricao
        FROM financeiro.situacao_titulo
        WHERE id > %s
        ORDER BY id;
    """

    pg_cur.execute(pg_query, (last_id,))
    rows = pg_cur.fetchall()

    print(f"Encontrados {len(rows)} novas situações de título.")

    # -----------------------------------------
    # 3. Inserção no SQL Server
    # -----------------------------------------
    inserted = 0

    for situacao_id, descricao in rows:
        sql_cur.execute("""
            INSERT INTO dw.DimSituacaoTitulo (
                SituacaoTituloID,
                Descricao
            )
            VALUES (?, ?)
        """, (situacao_id, descricao))

        inserted += 1

    # -----------------------------------------
    # 4. Commit e fechar conexões
    # -----------------------------------------
    sql_conn.commit()
    pg_conn.close()
    sql_conn.close()

    print(f"DimSituacaoTitulo - {inserted} registros inseridos com sucesso.")
    
load_dim_situacao_titulo()


=== Carregando DimSituacaoTitulo ===
Último SituacaoTituloID carregado: 0
Encontrados 4 novas situações de título.
DimSituacaoTitulo - 4 registros inseridos com sucesso.


In [22]:
# Carga DimVendedor

from database import get_postgres_cursor, get_sqlserver_cursor

def load_dim_vendedor():
    print("\n=== Carregando DimVendedor ===")

    # -----------------------------------------
    # Conexões
    # -----------------------------------------
    pg_conn, pg_cur = get_postgres_cursor()
    sql_conn, sql_cur = get_sqlserver_cursor()

    # -----------------------------------------
    # 1. Buscar maior VendedorID já carregado
    # -----------------------------------------
    sql_cur.execute("SELECT ISNULL(MAX(VendedorID), 0) FROM dw.DimVendedor;")
    last_id = sql_cur.fetchone()[0]

    print(f"Último VendedorID carregado: {last_id}")

    # -----------------------------------------
    # 2. Buscar novos vendedores no Postgres
    # -----------------------------------------
    pg_query = """
        SELECT DISTINCT 
            id_vendedor,
            vendedor
        FROM (
            SELECT 
                nf.id_vendedor,
                pfv.nome AS vendedor
            FROM vendas.nota_fiscal nf
            JOIN geral.pessoa_fisica pfv 
                ON pfv.id = nf.id_vendedor
        ) tb
        WHERE id_vendedor > %s
        ORDER BY id_vendedor ASC;
    """

    pg_cur.execute(pg_query, (last_id,))
    rows = pg_cur.fetchall()

    print(f"Encontrados {len(rows)} novos vendedores.")

    # -----------------------------------------
    # 3. Inserção no DW
    # -----------------------------------------
    inserted = 0

    for vendedor_id, vendedor_nome in rows:
        sql_cur.execute("""
            INSERT INTO dw.DimVendedor (
                VendedorID,
                Nome
            )
            VALUES (?, ?)
        """, (vendedor_id, vendedor_nome))

        inserted += 1

    # -----------------------------------------
    # 4. Commit & fechar conexões
    # -----------------------------------------
    sql_conn.commit()
    pg_conn.close()
    sql_conn.close()

    print(f"DimVendedor - {inserted} registros inseridos com sucesso.")

load_dim_vendedor()


=== Carregando DimVendedor ===
Último VendedorID carregado: 0
Encontrados 24 novos vendedores.
DimVendedor - 24 registros inseridos com sucesso.


In [23]:
# Carga FatoVendas

from database import get_postgres_cursor, get_sqlserver_cursor

def load_fato_vendas():
    print("\n=== Carregando FatoVendas ===")

    # -----------------------------------------
    # Conexões
    # -----------------------------------------
    pg_conn, pg_cur = get_postgres_cursor()
    sql_conn, sql_cur = get_sqlserver_cursor()

    # -----------------------------------------
    # 1. Buscar maior ID de item já carregado
    # -----------------------------------------
    sql_cur.execute("SELECT ISNULL(MAX(NotaFiscalKey), 0) FROM dw.FatoVendas;")
    last_nf_key = sql_cur.fetchone()[0]

    # Busca o maior ID do item_nf realmente carregado (melhor)
    sql_cur.execute("SELECT ISNULL(MAX(VendasKey), 0) FROM dw.FatoVendas;")
    last_vendas_key = sql_cur.fetchone()[0]

    print(f"Último VendasKey carregado: {last_vendas_key}")

    # -----------------------------------------
    # 2. Buscar novos itens no Postgres
    # -----------------------------------------
    pg_query = """
        SELECT 
            inf.id AS ItemID,
            inf.id_nota_fiscal,
            inf.id_produto,
            inf.quantidade,
            inf.valor_unitario,
            inf.valor_venda_real,
            p.valor_custo,
            nf.id_cliente,
            nf.id_vendedor,
            nf.id_forma_pagto,
            nf.data_venda,
            nf.numero_nf
        FROM vendas.item_nota_fiscal inf
        JOIN vendas.nota_fiscal nf ON nf.id = inf.id_nota_fiscal
        JOIN vendas.produto p ON p.id = inf.id_produto
        WHERE inf.id > %s
        ORDER BY inf.id;
    """

    pg_cur.execute(pg_query, (last_vendas_key,))
    rows = pg_cur.fetchall()

    print(f"Encontrados {len(rows)} novos itens para carga.")

    # -----------------------------------------
    # 3. Inserção no SQL Server
    # -----------------------------------------
    inserted = 0

    for (
        item_id,
        nota_id,
        produto_id,
        quantidade,
        valor_unitario,
        valor_venda_real,
        valor_custo,
        cliente_id,
        vendedor_id,
        forma_pagto_id,
        data_venda,
        numero_nf
    ) in rows:

        # -------------------------------
        # Lookup das Dimensões
        # -------------------------------

        # Produto
        sql_cur.execute("SELECT ProdutoKey FROM dw.DimProduto WHERE ProdutoID = ?", (produto_id,))
        produto_row = sql_cur.fetchone()
        if not produto_row:
            print(f"[WARN] ProdutoID {produto_id} não encontrado na DimProduto. Pulando.")
            continue
        produto_key = produto_row[0]

        # Cliente
        sql_cur.execute("SELECT ClienteKey FROM dw.DimCliente WHERE ClienteID = ?", (cliente_id,))
        cliente_row = sql_cur.fetchone()
        if not cliente_row:
            print(f"[WARN] ClienteID {cliente_id} não encontrado na DimCliente. Pulando.")
            continue
        cliente_key = cliente_row[0]

        # Vendedor
        sql_cur.execute("SELECT VendedorKey FROM dw.DimVendedor WHERE VendedorID = ?", (vendedor_id,))
        vendedor_row = sql_cur.fetchone()
        if not vendedor_row:
            print(f"[WARN] VendedorID {vendedor_id} não encontrado na DimVendedor. Pulando.")
            continue
        vendedor_key = vendedor_row[0]

        # Forma de pagamento
        sql_cur.execute("SELECT FormaPagamentoKey FROM dw.DimFormaPagamento WHERE FormaPagamentoID = ?", (forma_pagto_id,))
        fp_row = sql_cur.fetchone()
        if not fp_row:
            print(f"[WARN] FormaPagamentoID {forma_pagto_id} não encontrado na DimFormaPagamento. Pulando.")
            continue
        forma_pagto_key = fp_row[0]

        # Nota Fiscal
        sql_cur.execute("SELECT NotaFiscalKey FROM dw.DimNotaFiscal WHERE NotaFiscalID = ?", (nota_id,))
        nf_row = sql_cur.fetchone()
        if not nf_row:
            print(f"[WARN] NotaFiscalID {nota_id} não encontrado na DimNotaFiscal. Pulando.")
            continue
        nota_fiscal_key = nf_row[0]

        # Tempo (DateKey)
        date_key = int(data_venda.strftime("%Y%m%d"))

        # -------------------------------
        # Medidas calculadas
        # -------------------------------
        valor_custo_total = valor_custo * quantidade
        margem_bruta = valor_venda_real - valor_custo_total

        # -------------------------------
        # Inserção na FatoVendas
        # -------------------------------
        sql_cur.execute("""
            INSERT INTO dw.FatoVendas (
                DateKey,
                ProdutoKey,
                ClienteKey,
                VendedorKey,
                FormaPagamentoKey,
                NotaFiscalKey,
                NumeroNF,
                Quantidade,
                ValorUnitario,
                ValorVendaReal,
                ValorCustoTotal,
                MargemBruta
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            date_key,
            produto_key,
            cliente_key,
            vendedor_key,
            forma_pagto_key,
            nota_fiscal_key,
            numero_nf,
            quantidade,
            valor_unitario,
            valor_venda_real,
            valor_custo_total,
            margem_bruta
        ))

        inserted += 1

    # -----------------------------------------
    # 4. Commit e fechar
    # -----------------------------------------
    sql_conn.commit()
    pg_conn.close()
    sql_conn.close()

    print(f"FatoVendas - {inserted} registros inseridos com sucesso.")

load_fato_vendas()


=== Carregando FatoVendas ===
Último VendasKey carregado: 0
Encontrados 321720 novos itens para carga.
FatoVendas - 321720 registros inseridos com sucesso.


In [24]:
# Carga FatoRecebimentos

from database import get_postgres_cursor, get_sqlserver_cursor

def load_fato_recebimentos():
    print("\n=== Carregando FatoRecebimentos ===")

    # -----------------------------------------
    # 1. Conectar aos bancos
    # -----------------------------------------
    pg_conn, pg_cur = get_postgres_cursor()
    sql_conn, sql_cur = get_sqlserver_cursor()

    # -----------------------------------------
    # 2. Buscar maior RecebimentoID já carregado
    # -----------------------------------------
    sql_cur.execute("SELECT ISNULL(MAX(RecebimentoID), 0) FROM dw.FatoRecebimentos;")
    last_id = sql_cur.fetchone()[0]

    print(f"Último RecebimentoID carregado: {last_id}")

    # -----------------------------------------
    # 3. Buscar novos registros no Postgres
    # -----------------------------------------
    pg_query = """
        SELECT 
            cr.id AS RecebimentoID,
            cr.id_parcela AS ParcelaID,
            cr.valor_original,
            cr.valor_atual,
            cr.data_recebimento,
            cr.vencimento,
            cr.id_forma_pagamento,
            cr.id_situacao,
            p.id_nota_fiscal,
            nf.id_cliente
        FROM financeiro.conta_receber cr
        JOIN vendas.parcela p ON p.id = cr.id_parcela
        JOIN vendas.nota_fiscal nf ON nf.id = p.id_nota_fiscal
        WHERE cr.id > %s
        ORDER BY cr.id;
    """

    pg_cur.execute(pg_query, (last_id,))
    rows = pg_cur.fetchall()

    print(f"Encontrados {len(rows)} novos títulos a receber.")

    # -----------------------------------------
    # 4. Inserção linha a linha
    # -----------------------------------------
    inserted = 0

    for (
        rec_id,
        parcela_id,
        valor_original,
        valor_atual,
        data_recebimento,
        vencimento,
        forma_pagto_id,
        situacao_id,
        nota_fiscal_id,
        cliente_id
    ) in rows:

        # -------------------------------
        # LOOKUPS DAS DIMENSÕES
        # -------------------------------

        # Parcela
        sql_cur.execute("SELECT ParcelaKey FROM dw.DimParcela WHERE ParcelaID = ?", (parcela_id,))
        parcela_row = sql_cur.fetchone()
        if not parcela_row:
            print(f"[WARN] ParcelaID {parcela_id} não encontrada. Pulando.")
            continue
        parcela_key = parcela_row[0]

        # Cliente
        sql_cur.execute("SELECT ClienteKey FROM dw.DimCliente WHERE ClienteID = ?", (cliente_id,))
        cliente_row = sql_cur.fetchone()
        if not cliente_row:
            print(f"[WARN] ClienteID {cliente_id} não encontrado. Pulando.")
            continue
        cliente_key = cliente_row[0]

        # Forma Pagamento
        sql_cur.execute("SELECT FormaPagamentoKey FROM dw.DimFormaPagamento WHERE FormaPagamentoID = ?", (forma_pagto_id,))
        fp_row = sql_cur.fetchone()
        forma_pagto_key = fp_row[0] if fp_row else None

        # Situação Título
        sql_cur.execute("SELECT SituacaoTituloKey FROM dw.DimSituacaoTitulo WHERE SituacaoTituloID = ?", (situacao_id,))
        sit_row = sql_cur.fetchone()
        situacao_key = sit_row[0] if sit_row else None

        # Nota Fiscal
        sql_cur.execute("SELECT NotaFiscalKey FROM dw.DimNotaFiscal WHERE NotaFiscalID = ?", (nota_fiscal_id,))
        nf_row = sql_cur.fetchone()
        nota_fiscal_key = nf_row[0] if nf_row else None

        # -------------------------------
        # DATE KEYS
        # -------------------------------
        venc_key = int(vencimento.strftime("%Y%m%d"))

        if data_recebimento:
            receb_key = int(data_recebimento.strftime("%Y%m%d"))
            dias_atraso = (data_recebimento - vencimento).days
            valor_recebido = valor_atual
        else:
            receb_key = None
            dias_atraso = None
            valor_recebido = None

        # -------------------------------
        # INSERT NA FATO
        # -------------------------------
        sql_cur.execute("""
            INSERT INTO dw.FatoRecebimentos (
                RecebimentoID,
                ParcelaKey,
                ClienteKey,
                FormaPagamentoKey,
                SituacaoTituloKey,
                NotaFiscalKey,
                DataVencimentoKey,
                DataRecebimentoKey,
                ValorOriginal,
                ValorAtual,
                ValorRecebido,
                DiasEmAtraso
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            rec_id,
            parcela_key,
            cliente_key,
            forma_pagto_key,
            situacao_key,
            nota_fiscal_key,
            venc_key,
            receb_key,
            valor_original,
            valor_atual,
            valor_recebido,
            dias_atraso
        ))

        inserted += 1

    # -----------------------------------------
    # 5. Commit e fechar conexões
    # -----------------------------------------
    sql_conn.commit()
    pg_conn.close()
    sql_conn.close()

    print(f"FatoRecebimentos - {inserted} registros inseridos com sucesso.")
    
load_fato_recebimentos()


=== Carregando FatoRecebimentos ===
Último RecebimentoID carregado: 0
Encontrados 987022 novos títulos a receber.
FatoRecebimentos - 987022 registros inseridos com sucesso.


In [ ]:
# Carga FatoPagamentos

from database import get_postgres_cursor, get_sqlserver_cursor

def load_fato_pagamentos():
    print("\n=== Carregando FatoPagamentos ===")

    pg_conn, pg_cur = get_postgres_cursor()
    sql_conn, sql_cur = get_sqlserver_cursor()

    # Maior PagamentoID já carregado
    sql_cur.execute("SELECT ISNULL(MAX(PagamentoID), 0) FROM dw.FatoPagamentos;")
    last_id = sql_cur.fetchone()[0]
    print(f"Último PagamentoID carregado: {last_id}")

    # Query corrigida (sem join de fornecedor)
    pg_query = """
        SELECT 
            cp.id AS PagamentoID,
            cp.documento,
            cp.emissao,
            cp.vencimento,
            cp.data_pagamento,
            cp.valor_original,
            cp.valor_atual,
            cp.id_forma_pagamento,
            cp.id_situacao,
            cp.descricao
        FROM financeiro.conta_pagar cp
        WHERE cp.id > %s
        ORDER BY cp.id;
    """

    pg_cur.execute(pg_query, (last_id,))
    rows = pg_cur.fetchall()

    print(f"Encontrados {len(rows)} novos pagamentos.")

    inserted = 0

    for (
        pag_id,
        documento,
        emissao,
        vencimento,
        data_pagamento,
        valor_original,
        valor_atual,
        forma_pagto_id,
        situacao_id,
        descricao
    ) in rows:

        # ---- Dim Forma Pagamento ----
        sql_cur.execute("""
            SELECT FormaPagamentoKey 
            FROM dw.DimFormaPagamento 
            WHERE FormaPagamentoID = ?
        """, (forma_pagto_id,))
        fp_row = sql_cur.fetchone()
        forma_pagto_key = fp_row[0] if fp_row else None

        # ---- Dim Situação ----
        sql_cur.execute("""
            SELECT SituacaoTituloKey 
            FROM dw.DimSituacaoTitulo 
            WHERE SituacaoTituloID = ?
        """, (situacao_id,))
        sit_row = sql_cur.fetchone()
        situacao_key = sit_row[0] if sit_row else None

        # ---- Date Keys ----
        emissao_key = int(emissao.strftime("%Y%m%d"))
        venc_key = int(vencimento.strftime("%Y%m%d"))

        if data_pagamento:
            pagto_key = int(data_pagamento.strftime("%Y%m%d"))
            dias_atraso = (data_pagamento - vencimento).days
            valor_pago = valor_atual
        else:
            pagto_key = None
            dias_atraso = None
            valor_pago = None

        # ---- INSERT ----
        sql_cur.execute("""
            INSERT INTO dw.FatoPagamentos (
                PagamentoID,
                Documento,
                FormaPagamentoKey,
                SituacaoTituloKey,
                DataEmissaoKey,
                DataVencimentoKey,
                DataPagamentoKey,
                ValorOriginal,
                ValorAtual,
                ValorPago,
                DiasEmAtraso
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            pag_id,
            documento,
            forma_pagto_key,
            situacao_key,
            emissao_key,
            venc_key,
            pagto_key,
            valor_original,
            valor_atual,
            valor_pago,
            dias_atraso
        ))

        inserted += 1

    sql_conn.commit()
    pg_conn.close()
    sql_conn.close()

    print(f"FatoPagamentos - {inserted} registros inseridos com sucesso.")

load_fato_pagamentos()


=== Carregando FatoPagamentos ===
Último PagamentoID carregado: 0
Encontrados 3689 novos pagamentos.
FatoPagamentos - 3689 registros inseridos com sucesso.
